## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import os
import sys
sys.path.append(os.path.abspath('../'))
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Alice Springs,AU,-23.7000,133.8833,56.75,67,40,4.61,scattered clouds
1,1,Kodiak,US,57.7900,-152.4072,40.51,81,100,17.27,light rain
2,2,Grande Prairie,CA,55.1667,-118.8027,41.18,93,100,5.75,overcast clouds
3,3,Warmbad,NaN,-28.4500,18.7333,70.20,14,0,10.83,clear sky
4,4,Pozo Colorado,PY,-23.4923,-58.7972,88.39,45,0,4.90,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like? "))
max_temp = float(input("What is the maximum temperature you would like? "))

What is the minimum temperature you would like? 75
What is the maximum temperature you would like? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
matching_city_df.count()

City_ID                219
City                   219
Country                218
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
vacation_loc_df = matching_city_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_loc_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Pozo Colorado,PY,88.39,clear sky,-23.4923,-58.7972,
6,Alofi,NU,80.92,overcast clouds,-19.0595,-169.9187,
7,Kununurra,AU,85.98,clear sky,-15.7667,128.7333,
20,Biak,ID,76.73,broken clouds,-0.9131,122.8766,
25,Sabang,ID,84.06,overcast clouds,5.8933,95.3214,
27,Sao Joao Da Barra,BR,79.83,clear sky,-21.6403,-41.0511,
30,Chicama,PE,76.62,broken clouds,-7.8447,-79.1469,
38,Carutapera,BR,84.24,scattered clouds,-1.1950,-46.0200,
39,Havelock,US,80.98,broken clouds,34.8791,-76.9013,
40,Natal,BR,84.85,broken clouds,-5.7950,-35.2094,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", None, inplace = True)
hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. .
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))